# Predicate Logic Solution

Enter Premises and a Query in the boxes below.
____________________________________________________________

Sample Premises: All living things are plants or animals. Rama's dog is a living thing. Rama's dog is not a plant. All animals have a heart.

Sample Query: Rama's dog has a heart.

Output: Query is TRUE.
____________________________________________________________

Sample Premises: All boys are humans. Some humans are good.

Sample Query: Some boys are good.

Output: Query is TRUE.



In [55]:
import spacy
import pandas as pd
from ipywidgets import *

nlp = spacy.load("en_core_web_sm")

In [18]:
def extractPremise(query):
    q=nlp(query)
    q_sub=""
    q_pos=True
    q_pred=""
    q_quant=""
    q_whoThat=False
    q_verb=""
    for token in q:
        if token.lemma_ in ["all","every","everyone","each","everything"]:
            q_quant="Universal"
        if token.lemma_ in ["some","atleast","someone"]:
            q_quant="Existential"
        if token.lemma_ in ["who","that"]:
            q_whoThat=True
        if token.dep_=="relcl" and q_whoThat==True:
            q_sub+=token.lemma_
        if token.dep_=="dobj" and q_whoThat==True:
            q_sub+=" "+token.lemma_
        if token.dep_=="advmod" and q_whoThat==True:
            q_sub+=" "+token.lemma_
        if token.dep_=="nsubj" and not (token.lemma_ in ["someone","something","everyone","everything","who","that"]):
            subject_compound=""
            for child in token.children:
                if child.dep_=="compound":
                    subject_compound+=child.lemma_+" "
                if child.dep_=="amod":
                        subject_compound+=child.lemma_+" "
                if child.dep_=="poss":
                    subject_compound+=child.lemma_+" "
            subject_compound+=token.lemma_
            q_sub=subject_compound
        if token.dep_=="acomp":
                q_pred=token.lemma_
        if token.dep_=="ROOT":
            if token.pos_=="NOUN":
                q_pred+=token.lemma_+" "
            if token.pos_=="VERB":
                q_pred+=token.lemma_+" "
                q_verb=token.lemma_+" "
            for child in token.children:
                if child.dep_=="dobj":
                    q_pred+=child.lemma_+" "
                if child.dep_=="attr":
                    subject_compound=""
                    for child2 in child.children:
                        if child2.dep_=="compound":
                            subject_compound+=child2.lemma_+" "
                        if child2.dep_=="amod":
                                subject_compound+=child2.lemma_+" "
                        if child2.dep_=="poss":
                            subject_compound+=child2.lemma_+" "
                    subject_compound+=child.lemma_
                    q_pred=subject_compound
                if child.dep_=="advmod":
                    q_pred+=child.lemma_

        if token.lemma_=="no" or token.dep_=="neg":
            q_pos=False
    return q_quant.strip(), q_sub.lower().strip(), q_pos, q_pred.lower().strip(), q_verb

In [22]:
def solvePred(premise,query):
    if premise=="" or query=="":
        return("Empty Premise or Query! Try again.")
    quantifier=[]
    subject=[]
    positive=[]
    predicate=[]
    conjugation=[]
    predicate2=[]
    verb=[]
    doc=nlp(premise)
    
    sentences=[]
    for sent in doc.sents:
        sentences.append(sent.text)
    for sent in sentences:
        sentence=nlp(sent)
        quant=""
        sub=""
        pos=True
        pred=""
        conj=""
        pred2=""
        sent_verb=""
        whoThat=False
        for token in sentence:
            if token.lemma_ in ["who", "that"]:
                whoThat=True
            if token.lemma_ in ["all","every","everyone","each","everything"]:
                quant="Universal"
            if token.lemma_ in ["some","atleast","someone"]:
                quant="Existential"
            if token.lemma_ == "or":
                conj=token.lemma_
            if token.dep_=="conj" and conj=="or":
                pred2=sent_verb+token.lemma_
            if token.lemma_ =="and":
                conj=token.lemma_
                andindex=sent.find(" and ")
                pred2=sent[andindex+5:]
                break
            if token.dep_=="relcl" and whoThat==True:
                sub+=token.lemma_
            if token.dep_=="dobj" and whoThat==True:
                sub+=" "+token.lemma_
            if token.dep_=="advmod" and whoThat==True:
                sub+=" "+token.lemma_
            if token.dep_=="nsubj" and not (token.lemma_ in ["someone","something","everyone","everything","who","that"]):
                subject_compound=""
                for child in token.children:
                    if child.dep_=="compound":
                        subject_compound+=child.lemma_+" "
                    if child.dep_=="amod":
                        subject_compound+=child.lemma_+" "
                    if child.dep_=="poss":
                        subject_compound+=child.lemma_+" "
                subject_compound+=token.lemma_
                sub=subject_compound
            if token.dep_=="acomp":
                pred=token.lemma_
            if token.dep_=="ROOT":
                if token.pos_=="VERB":
                    pred+=token.lemma_+" "
                    sent_verb=token.lemma_+" "
                for child in token.children:
                    if child.dep_=="dobj":
                        pred+=child.lemma_+" "
                    if child.dep_=="attr":
                        subject_compound=""
                        for child2 in child.children:
                            if child2.dep_=="compound":
                                subject_compound+=child2.lemma_+" "
                            if child2.dep_=="amod":
                                subject_compound+=child2.lemma_+" "
                            if child2.dep_=="poss":
                                subject_compound+=child2.lemma_+" "
                        subject_compound+=child.lemma_
                        pred=subject_compound
                    if child.dep_=="advmod":
                        pred+=child.lemma_

            if token.lemma_=="no" or token.dep_=="neg":
                pos=False
        quantifier.append(quant.strip())
        subject.append(sub.lower().strip())
        positive.append(pos)
        predicate.append(pred.lower().strip())
        conjugation.append(conj.lower().strip())
        predicate2.append(pred2.lower().strip())
        verb.append(sent_verb)
        
    data = {'Quantifier':quantifier,
        'Subject':subject,
       'Positive':positive,
       'Predicate':predicate,
       'And/Or':conjugation,
       'Second Predicate':predicate2}
 
    # Create knowledge Base
    knowledgeBase = pd.DataFrame(data)

    
    ## Resolving AND
    
    for i in range (len(conjugation)):
        if conjugation[i]=="and":
            conjugation[i]=""
            q_quant_new, q_sub_new, q_pos_new, q_pred_new, q_verb_new = extractPremise(predicate2[i])
            predicate2[i]=""
            if(q_sub_new==""): #if only phrase is there after and
                subject.append(subject[i])
                positive.append(q_pos_new)
                predicate.append(verb[i]+q_pred_new)
                quantifier.append(quantifier[i])
                conjugation.append("")
                predicate2.append("")
                verb.append(verb[i])
            else: #if independent sentence after and
                subject.append(q_sub_new)
                positive.append(q_pos_new)
                predicate.append(q_pred_new)
                quantifier.append(q_quant_new)
                conjugation.append("")
                predicate2.append("")
                verb.append(q_verb_new)
            
    
    data = {'Quantifier':quantifier,
        'Subject':subject,
       'Positive':positive,
       'Predicate':predicate,
        'And/Or':conjugation,
       'Second Predicate':predicate2}
    
    # Create knowledge Base
    ANDResolvedKnowledgeBase = pd.DataFrame(data)

    
    # Extending Knowledge Base
    for i in range (len(predicate)):
        for j in range (len(subject)):
            # Modus Ponens
            if predicate[i]==subject[j]:#add ignore case
                subject.append(subject[i])
                positive.append((positive[i] and positive[j]))
                predicate.append(predicate[j])
                conjugation.append(conjugation[j])
                predicate2.append(predicate2[j])
                if(quantifier[i]=="Universal"==quantifier[j]):
                    quantifier.append("Universal")
                elif(quantifier[i]=="Existential"==quantifier[j]):
                    quantifier.append("Existential")
                elif((quantifier[i]=="Universal" and quantifier[j]=="Existential") or (quantifier[j]=="Universal" and quantifier[i]=="Existential")):
                    quantifier.append("Existential")
                else:
                    quantifier.append("")
            # Modus Tollens
            if predicate[i]==predicate[j]:
                if conjugation[i]==conjugation[j]=="":
                    if positive[i] == True and positive[j] == False:
                        subject.append(subject[j])
                        positive.append(False)
                        predicate.append(subject[i])
                        conjugation.append("")
                        predicate2.append("")
                        if(quantifier[i]=="Universal"==quantifier[j]):
                            quantifier.append("Universal")
                        elif(quantifier[i]=="Existential"==quantifier[j]):
                            quantifier.append("Existential")
                        elif((quantifier[i]=="Universal" and quantifier[j]=="Existential") or (quantifier[j]=="Universal" and quantifier[i]=="Existential")):
                            quantifier.append("Existential")
                        else:
                            quantifier.append("")
                
    data = {'Quantifier':quantifier,
        'Subject':subject,
       'Positive':positive,
       'Predicate':predicate,
        'And/Or':conjugation,
       'Second Predicate':predicate2}
 
    # Create knowledge Base
    extendedKnowledgeBase = pd.DataFrame(data)

    
    # Resolving OR in Knowledge Base
    for i in range (len(predicate)):
        for j in range (len(subject)):
            if subject[i]==subject[j]:
                if predicate[i]==predicate[j] or predicate[i]==predicate2[j]:
                    if conjugation[j]=="or":
                        if(positive[i]==True and positive[j]==False):
                            print()
                            print("Inconsistent premises:\n")
                            print(subject[i],positive[i],predicate[i])
                            print(subject[j],positive[j],predicate[j], conjugation[j], predicate2[j])
                        elif(positive[j]==True and positive[i]==False):
                            subject.append(subject[i])
                            positive.append(True)
                            conjugation.append("")
                            predicate2.append("")
                            if predicate[i]==predicate[j]:
                                predicate.append(predicate2[j])
                            else:
                                predicate.append(predicate[j])
                            if(quantifier[i]=="Universal"==quantifier[j]):
                                quantifier.append("Universal")
                            elif(quantifier[i]=="Existential"==quantifier[j]):
                                quantifier.append("Existential")
                            elif((quantifier[i]=="Universal" and quantifier[j]=="Existential") or (quantifier[j]=="Universal" and quantifier[i]=="Existential")):
                                quantifier.append("Existential")
                            else:
                                quantifier.append("")

    data = {'Quantifier':quantifier,
        'Subject':subject,
       'Positive':positive,
       'Predicate':predicate,
        'And/Or':conjugation,
       'Second Predicate':predicate2}
 
    # Create knowledge Base
    extendedKnowledgeBase = pd.DataFrame(data)

    # Print the output.
    print("\n OR Resolved Knowledge Base:")
    print(extendedKnowledgeBase)
    
    # Extending OR Resolved Knowledge Base
    for i in range (len(predicate)):
        for j in range (len(subject)):
            # Modus Ponens
            if predicate[i]==subject[j]:
                subject.append(subject[i])
                positive.append((positive[i] and positive[j]))
                predicate.append(predicate[j])
                conjugation.append(conjugation[j])
                predicate2.append(predicate2[j])
                if(quantifier[i]=="Universal"==quantifier[j]):
                    quantifier.append("Universal")
                elif(quantifier[i]=="Existential"==quantifier[j]):
                    quantifier.append("Existential")
                elif((quantifier[i]=="Universal" and quantifier[j]=="Existential") or (quantifier[j]=="Universal" and quantifier[i]=="Existential")):
                    quantifier.append("Existential")
                else:
                    quantifier.append("")
            # Modus Tollens
            if predicate[i]==predicate[j]:
                if conjugation[i]==conjugation[j]=="":
                    if positive[i] == True and positive[j] == False:
                        subject.append(subject[j])
                        positive.append(False)
                        predicate.append(subject[i])
                        conjugation.append("")
                        predicate2.append("")
                        if(quantifier[i]=="Universal"==quantifier[j]):
                            quantifier.append("Universal")
                        elif(quantifier[i]=="Existential"==quantifier[j]):
                            quantifier.append("Existential")
                        elif((quantifier[i]=="Universal" and quantifier[j]=="Existential") or (quantifier[j]=="Universal" and quantifier[i]=="Existential")):
                            quantifier.append("Existential")
                        else:
                            quantifier.append("")
                
    data = {'Quantifier':quantifier,
        'Subject':subject,
       'Positive':positive,
       'Predicate':predicate,
        'And/Or':conjugation,
       'Second Predicate':predicate2}
 
    # Create knowledge Base
    extendedKnowledgeBase = pd.DataFrame(data)
    extendedKnowledgeBase.drop_duplicates(subset =['Quantifier','Subject','Positive','Predicate','And/Or','Second Predicate'], 
                     keep = 'last', inplace = True) 
    
    q_quant, q_sub, q_pos, q_pred, q_verb = extractPremise(query)
    for i in range (len(predicate)):
        if subject[i]==q_sub and predicate[i]==q_pred:
            if quantifier[i]==q_quant:
                if positive[i]==q_pos:
                    return("\nQuery is TRUE.")
                else:
                    return("\nQuery is FALSE.")
            
    return("\nInsufficient data. Query cannot be determined as TRUE or FALSE.")

In [54]:
txt_layout = Layout(width='550px')
premise_box = Text(description='Premise', layout=txt_layout, value="")
query_box = Text(description='Query', layout=txt_layout, value="")

go_btn = Button(description='GO', button_style='success', layout=Layout(width='50px'))

result="Enter Premise and Query in the boxes above."
sol_lbl = Label()
sol_lbl.value=result
sol_lbl.layout.width = '1000px'


def optimize():
    result=solvePred(premise_box.value,query_box.value)
    sol_lbl.value=result
    
go_btn.on_click(lambda btn: optimize())

In [53]:
VBox([premise_box, query_box, go_btn, sol_lbl])